# Autori

* Rossi Giorgio, matricola: 928570
* Repetto Valeria, matricola: 939740

# Sommario

Sulla base del tutorial https://www.tensorflow.org/tutorials/generative/dcgan#what_are_gans, nel seguente progetto è stato deciso di utilizzare un dataset differente per generare volti umani, utilizzando immagini disponibili al link(https://www.kaggle.com/jessicali9530/celeba-dataset).

# DC-GAN

Le GAN (Generative Adversial Networks) sono dei network generativi costituiti da due modelli, il generatore e il discriminatore, rappresentati da due reti neurali differenti, che vengono allenate simultaneamente in maniera avversa: il generatore impara a produrre dei dati(generalmente immagini, video o audio) che somiglino più possibile a quelli reali mentre il discriminatre impara a distinguere i dati reali da quelli falsi. 
In un sistema così costituito il generatore imparera a creare,a partire da un rumore iniziale, dati con l'obbiettivo di "ingannare" il discriminatore, e quest'ultimo "guida" la sua contro parte a creare immaginisempre più realistiche.

Le DC GAN(Deep Convolutional GAN) utilizzano delle tecniche del deep learning nel training delle GANs. Queste sono principalmente i Convolutional layers al fine di aumentare e diminuire la dimensione spaziale delle features del problema, e la BatchNormalization che viene utilizzata per normalizzare i features vectors a media e varianza unitaria in ogni layer. Queste servono a dare stabilità al problema di learning.

> # Estrazione delle immagini dal dataset

Le immagini sono state tagliate in maniera tale da ottenere solo l'ovale del viso, con l'obiettivo di ridurre le informazioni inutili e quindi rumore. In seguito, per rapidità computazionale, le immagini sono state portate ad una dimensione di (28x28x1), eliminando dunque i colori e tenendo solo la scala dei grigi ed i valori dei pixel sono stati normalizzati tra -1 e 1.

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
from tensorflow.keras import layers
import time
import tensorflow as tf
from IPython import display

path_celeb = []
train_path_celeb = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/"
print(len(os.listdir(train_path_celeb)))

for path in os.listdir(train_path_celeb):
    if '.jpg' in path:
        path_celeb.append(os.path.join(train_path_celeb, path))
       
new_path=path_celeb[0:40000]
crop = (30, 55, 150, 175) #croping size for the image so that only the face at centre is obtained
#images = [np.array((Image.open(path).crop(crop)).resize((28,28))) for path in new_path]
#images = [np.array((Image.open(path)).resize((28,28))) for path in new_path]

images = [np.array((Image.open(path).crop(crop)).resize((64,64))) for path in new_path]
print('min e max',np.array(images).min(),np.array(images).max())
plt.imshow(images[3])

"""
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray
for i in range(len(images)):
    images[i]=rgb2gray(images[i])
    

    
print(np.array(images).shape)
 """
for i in range(len(images)):
    images[i] = ((images[i] - images[i].min())/(255 - images[i].min()))
    images[i] = images[i]*2-1
    
images = np.array(images)
train_images = images.reshape(images.shape[0], 64, 64, 3).astype('float32')
print('min e max',train_images.min(),train_images.max())

print(images.shape)

Di tutte le immagini disponibli è stato preso un campione di 50000 foto.

In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

> # Creazione modelli

> > # Generatore

Il generatore è in grado di creare un immagine a partire da un rumore random. Il modello è costruito a partire da uno strato Dense, che prende il rumore in input, e successivamente esegue un processo di upsampling successivi fino a raggiungere un immagine delle dimensioni desiderate, nel nostro caso(28X28X1). Per ogni strato, ad eccezione di quello finale che usa tanh, la funzione di attivazione usata è stata la LeakyReLU, largamente utilizzata nelle reti neurali perchè ovvia il problema di saturazione del gradiente.

In [ ]:

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(4*4*1024, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((4, 4, 1024)))
    assert model.output_shape == (None, 4, 4, 1024) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(512,(5,5), strides=(1, 1), padding='same', use_bias=False))
    #assert model.output_shape == (None, 8, 8, 512)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(256, (5,5), strides=(2, 2), padding='same', use_bias=False))
    #assert model.output_shape == (None, 16, 16, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    
    model.add(layers.Conv2DTranspose(128, (5,5), strides=(2, 2), padding='same', use_bias=False))
   # assert model.output_shape == (None, 32, 32, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64, (5,5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3,(5,5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    
    return model

Di seguito un esempio di immagine generata a partire da un rumore random

In [ ]:
generator = make_generator_model()



In [ ]:
samples = 10
x_fake = generator.predict(np.random.normal(loc=0, scale=1, size=(samples,100)))
print(x_fake.shape)
for k in range(samples):
    plt.subplot(2, 5, k+1)
    plt.imshow(x_fake[k].reshape(64,64,3))
    plt.xticks([])
    plt.yticks([])

        
plt.tight_layout()
plt.show()

> > # Discriminatore

Il discriminatore segue il medello di una rete per classificazione di immagini con convolutional neural networks.
Il modello verrà allenato in modo da classificare con un valore positivo le immagini reali, e conseguentemente con un numero negativo le immagini false.

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32,(5,5), strides=(2, 2), padding='same',
                                     input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64,(5,5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128,(5,5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(256,(5,5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

discriminator = make_discriminator_model()
#decision = discriminator(generated_image)
#print (decision)


> # Loss e ottimizzazione

Il fulcro del funzionamento delle G.A.N. sta, come suggerisce il nome, nella competizione tra generatore e discriminatore, pertanto ciascuno dei due avra la propria loss function così come la propria funzione di ottimizzazione.

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

> > # Loss Discriminatore

La loss function del discriminatore deve dare una stima di quanto il modello usato riesca a distinguere immagini di volti umani  vere da quelle false. Per ottenere ciò la loss confronta la classificazione di immagini vere eseguita dal discriminatore con un array di 1, mentre la decisione presa dallo stesso modello per le immagini create dal generatore verrà confrontata con un array di 0.

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

> > # Loss Generatore

Contrariamente alla sua controparte la loss function del generatore deve riuscire a quantificare quanto bene il modello riesce a "imbrogliare" il discriminatore, pertanto la classificazione fatta dal discriminatore viene confrontata con un array di 1.
In questo caso il goal del modello è massimizzare la loss.

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

> > # Ottimizazione

Per i due modelli sono state create due ottimizzazioni diverse, per le ragioni già esposte sopra.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5)

Di seguito è mostrato un chekpoint.

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# Training loop

Il ciclo di training parte da un rumore random che viene dato in pasto al generatore, che lo usa per dare alla luce un'immagine; il discriminatore quindi procede alla classificazione di un'immagine vera, estratta dal data set, e dell'immagine generata in precedenza. 
Per ciascun modello viene calcolata la loss, e conseguentemente il gradiente attravero cui poi i due modelli vengono rispettivamente aggiornati.

In [ ]:
EPOCHS = 300
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
seed = tf.random.normal([num_examples_to_generate, noise_dim])
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return gen_loss, disc_loss

Per ogni step di training è stata salvata la loss di discriminatore e generatore al fine di studiarne l'andamento al variare dell'epoch

In [ ]:
def train(dataset, epochs):
  D_loss=[] #list to collect loss for the discriminator model
  G_loss=[] #list to collect loss for generator model  
  for epoch in range(epochs):
    start = time.time()
    cont=0
    gen_mediandum=[]
    disc_mediandum=[]
    for image_batch in dataset:
      cont+=1  
      gen_loss, disc_loss=train_step(image_batch)
      
      gen_mediandum.append(gen_loss)
      disc_mediandum.append(disc_loss)
      if (cont==int(BUFFER_SIZE/BATCH_SIZE)+1):
          
          G_loss.append(np.array(gen_mediandum).mean()) #list to collect loss for generator model 
          D_loss.append(np.array(disc_mediandum).mean()) #list to collect loss for the discriminator model
          gen_mediandum=[]
          disc_mediandum=[]
    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)
  return G_loss,D_loss

In [ ]:
def tensor_to_image(tensor):
  #tensor = tensor
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [ ]:

def generate_and_save_images(model, epoch, test_input):
  if ((epoch<85) or (epoch>298)):  
      # Notice `training` is set to False.
      # This is so all layers run in inference mode (batchnorm).
      predictions = model(test_input, training=False)

      fig = plt.figure(figsize=(4,4))

      for i in range(predictions.shape[0]):
          plt.subplot(4, 4, i+1)
          plt.imshow(predictions[i,:,:,0] )
         
          plt.axis('off')
      plt.suptitle('Image at epoch: %i' %epoch)    
      plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
      plt.show()
    
G_loss,D_loss=train(train_dataset, EPOCHS)
print('vabene2')

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))
display_image(EPOCHS)

# Studio delle funzioni di perdita

Per trovare il numero ottimale di epoch da utilizzare, sono stati inseriti in un grafico le funzioni di loss del generatore e del discriminatore e la loro differenza(diff_loss).

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(G_loss,color='red',label='Generator_loss')
plt.plot(D_loss,color='blue',label='Discriminator_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss') 
plt.title('Model loss per epoch')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
diff_loss=(np.array(G_loss)-np.array(D_loss))
plt.plot(diff_loss,color='green',label='Diff_loss')
print("number of epoch for max diff",max(diff_loss))
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('Model loss per epoch')
plt.show()

L'algoritmo di ottimizzazione ricerca i valori dei parametri che massimizzino la loss del generatore ed al tempo stesso minimizzino quella del discriminatore. Studiandone la differenza è possibile rendersi conto di come il numero ottimale di epoch si aggiri intorno a 80-85 e di come, man mano che il numero aumenta, avvenga il fenomeno dell'overfitting: questo è stato riscontrato negli esempi generati, dato che le immagini corrispondenti ad alte epoch(esempio 300) sono meno accurate e verosimili ad occhio umano rispetto quelle generate ed epoch ottimale(80)

# Referenze

* https://www.tensorflow.org/tutorials/generative/dcgan#what_are_gans
* https://www.kaggle.com/sayakdasgupta/fake-faces-with-dcgans#Reference
* https://www.freecodecamp.org/news/an-intuitive-introduction-to-generative-adversarial-networks-gans-7a2264a81394/